In [1]:
import pandas as pd
import numpy as np
import nltk 
from bs4 import BeautifulSoup            
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
import math
from textblob import TextBlob as tb
from sklearn.pipeline import Pipeline


In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head()


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

18.362078428268433


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...","{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...","{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...","{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...","{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]","{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
def getDataFrameFilterByToxicClass(df, toxicName, cls = 1):
    return train[train[toxicName] == cls]['listOfCleanWords']

toxicName = 'toxic'

clnTxtWrds = getDataFrameFilterByToxicClass(train, toxicName).tolist()
clnTxtWrdsNoToxic = getDataFrameFilterByToxicClass(train, toxicName, 0).tolist()

allTextsToxic = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]
allTextsNoToxic = [' '.join(clnTxtWrdsNoToxic[x]) for x in range(len(clnTxtWrdsNoToxic))]

In [6]:
allTextsToxic[:10]

['cocksucker piss around work',
 'hey talk exclusive group wp taliban good destroying self appointed purist gang one asks question abt anti social destructive non contribution wp ask sityush clean behavior issue nonsensical warning',
 'bye look come think comming back tosser',
 'gay antisemmitian archangel white tiger meow greetingshhh uh two way erased comment ww holocaust brutally slaying jew gay gypsy slav anyone anti semitian shave head bald go skinhead meeting doubt word bible homosexuality deadly sin make pentagram tatoo forehead go satanistic mass gay pal first last warning fucking gay appreciate nazi shwain would write page wish talk anymore beware dark side',
 'fuck filthy mother as dry',
 'sorry sorry screwed around someone talk page bad know template talk page help assert dominance know bow almighty administrator going go play outside mom',
 'get fucked get fuckeeed got drink cant put get fuck get fucked fucked right',
 'stupid peace shit stop deleting stuff asshole go die f

In [7]:
allTextsNoToxic[:10]

['explanation edits made username hardcore metallica fan reverted vandalism closure gas voted new york doll fac please remove template talk page since retired',
 'aww match background colour seemingly stuck thanks talk january utc',
 'hey man really trying edit war guy constantly removing relevant information talking edits instead talk page seems care formatting actual info',
 'make real suggestion improvement wondered section statistic later subsection type accident think reference may need tidying exact format ie date format etc later one else first preference formatting style reference want please let know appears backlog article review guess may delay reviewer turn listed relevant form eg wikipedia good article nomination transport',
 'sir hero chance remember page',
 'congratulation well use tool well talk',
 'vandalism matt shirvington article reverted please banned',
 'sorry word nonsense offensive anyway intending write anything article wow would jump vandalism merely requestin

In [8]:
topNFeatures = 500
tdidfVectTrainToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainToxic = tdidfVectTrainToxic.fit_transform(allTextsToxic)

featureArrayToxicTrain = np.array(tdidfVectTrainToxic.get_feature_names())
tfidfOrderedToxicTrain = np.argsort(matrixTdidfTrainToxic.toarray()).flatten()[::-1]


tdidfVectTrainNoToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainNoToxic = tdidfVectTrainNoToxic.fit_transform(allTextsNoToxic)

featureArrayNoToxicTrain = np.array(tdidfVectTrainNoToxic.get_feature_names())
tfidfOrderedNoToxicTrain = np.argsort(matrixTdidfTrainNoToxic.toarray()).flatten()[::-1]




In [9]:
selectedWordsTrainToxic = featureArrayToxicTrain[tfidfOrderedToxicTrain][:topNFeatures]
selectedWordsTrainNoToxic = featureArrayNoToxicTrain[tfidfOrderedNoToxicTrain][:topNFeatures]
selectedWordsTrain = []
for x in range(topNFeatures):
    if len(selectedWordsTrain) <= topNFeatures: 
        selectedWordsTrain = np.unique(np.append(selectedWordsTrain, np.unique([selectedWordsTrainNoToxic[x], selectedWordsTrainToxic[x]])))    
selectedWordsTrain = selectedWordsTrain[:topNFeatures]


In [10]:
len(selectedWordsTrain)

500

In [11]:
# With all selected toxic words get BoW from all Texts 

clnTxtWrds = train['listOfCleanWords']
allTexts = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]

vectTrain = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTrain = vectTrain.fit_transform(allTexts).toarray()

In [12]:
matrixBoWTrain.shape

(159571, 500)

In [13]:
matrixBoWTrain[6]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [14]:
pd.DataFrame(matrixBoWTrain, columns=selectedWordsTrain).head()

,abusing,according,account,act,action,actually,add,added,adding,addition,...,whole,whore,wiki,wikipedia,wikiproject,wish,within,without,woman,word
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
matrixBoWTrain

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
X = matrixBoWTrain
y = np.array(train['toxic'])
Nfold = 5
skf = StratifiedKFold(n_splits=Nfold, shuffle=True)
skf.get_n_splits(X, y)
meanAcc = 0.0
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    nb = MultinomialNB()
    %time nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    meanAcc += acc
    print (acc)

print("La precisión media es de:", np.round(meanAcc / Nfold, 4))



Wall time: 400 ms
0.948331505562
Wall time: 396 ms
0.947046843177
Wall time: 406 ms
0.945885818136
Wall time: 405 ms
0.947797204988
Wall time: 402 ms
0.948140256322
La precisión media es de: 0.9474


In [17]:
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()

,Unnamed: 0,id,comment_text,listOfCleanWords
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,"['yo', 'bitch', 'ja', 'rule', 'succesful', 'ev..."
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"['rfc', 'title', 'fine', 'imo']"
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...","['source', 'zawe', 'ashton', 'lapland']"
3,3,00017563c3f7919a,":If you have a look back at the source, the in...","['look', 'back', 'source', 'information', 'upd..."
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,"['anonymously', 'edit', 'article']"


In [18]:
clnTxtWrdsTest = test['listOfCleanWords'].tolist()
allTextsTest = [' '.join(clnTxtWrdsTest[x]) for x in range(len(clnTxtWrdsTest))]

In [19]:
vectTest = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTest = vectTest.fit_transform(allTextsTest).toarray()

In [20]:
matrixBoWTest.shape

(153164, 500)

In [21]:
X_test = matrixBoWTest

In [22]:
nb = MultinomialNB()
%time nb.fit(X, y)
y_pred = nb.predict(X_test)


Wall time: 540 ms


In [23]:
np.where(y_pred > 0)

(array([], dtype=int64),)

In [24]:
X.shape

(159571, 500)